In [5]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import copy
import os
import random
random.seed(555)  

# pathways process and network generation

In [6]:
#Read the pathway feature generated by GCN
pathway_feature = pd.read_csv('./data/Pathways_Feature.csv',index_col =0)

#pathway data loading
data = pd.read_csv('./data/ReactomePathwaysRelation_new_download.txt',sep = '\t',header=None)

data.columns = ['child','parent']
human_hierarchy = data[data['child'].str.contains('HSA')]  

# construct pathway graph
net = nx.from_pandas_edgelist(human_hierarchy, 'child', 'parent', create_using=nx.DiGraph()) 
net.name = 'reactome'

roots = [n for n, d in net.in_degree() if d == 0]  
root_node = 'root'
edges = [(root_node, n) for n in roots] 
net.add_edges_from(edges)  

In [7]:
#Get the node of the current layer
def get_nodes_at_level(net, distance):
    nodes = set(nx.ego_graph(net, 'root', radius=distance))
    if distance >= 1.:
        nodes -= set(nx.ego_graph(net, 'root', radius=distance - 1))
    return list(nodes)

def get_nodes(net,num):
    net_nodes = [] 
    
    for i in range(1,num+1):
        net_nodes.append(get_nodes_at_level(net,i))
        
    return net_nodes

#  define network size. for example,this network is five layer
net_num = 5
net_nodes = get_nodes(net,net_num)

In [8]:
#  Read the connection of a single node to the next layer
def add_node(net,net_nodes):
        
    for i in range(len(net_nodes)-2,-1,-1):
        
        data_temp = copy.deepcopy(net_nodes[i])
        
        for n in net_nodes[i]:
            nexts = net.successors(n)         
            temp = [ nex  for nex in nexts ] 
            if len(temp)==0:
                data_temp.remove(n)  # If the node of the current layer has no successor node, remove the node
            elif len(set(temp).intersection(set(net_nodes[i+1])))==0:   #if the subsequent node of the node of the current layer is not on the next layer, delete the node
                data_temp.remove(n)
            else:
                continue
        net_nodes[i] = data_temp
    return net_nodes

net_nodes  =  add_node(net,net_nodes)

In [9]:

def get_note_relation(net_nodes):
    node_mat = []
   
    for i in range(len(net_nodes)-1):
        dicts = {}
        for n in net_nodes[i]:
            nexts = net.successors(n)  
            x = [ nex   for nex in nexts if nex in net_nodes[i+1] ]
            dicts[n] = x

        mat = np.zeros((len(net_nodes[i]), len(net_nodes[i+1]))) 
        for p, gs in dicts.items():     
            g_inds = [net_nodes[i+1].index(g) for g in gs]
            p_ind = net_nodes[i].index(p)
            mat[p_ind, g_inds] = 1

        df = pd.DataFrame(mat, index=net_nodes[i], columns=net_nodes[i+1])
        node_mat.append(df.T)
    return node_mat

Get_Node_relation = get_note_relation(net_nodes)

In [10]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

In [11]:
#Link prediction only for adjacent pathways
for k in range(0,4):
    for col in Get_Node_relation[k].columns:
        if col in list(pathway_feature.index):
            for row in Get_Node_relation[k].index:
                if row in list(pathway_feature.index):
                    pos_c = pathway_feature.loc[col].values
                    pos_r = pathway_feature.loc[row].values
                    score = sigmoid(pos_c.dot(pos_r))

                    if score >0.9:  #The similarity is greater than 0.9
                        Get_Node_relation[k][col][row] = 1

In [12]:
np.array(Get_Node_relation[3].values.nonzero()).shape

(2, 1745)

In [13]:
# Read gene-pathway annotation relationships
import re
def load_data_dict(filename):

    data_dict_list = []
    dict = {}
    with open( filename) as gmt:
        data_list = gmt.readlines()

        # print data_list[0]
        for row in data_list:
            genes = row.split('\t')
            
            genes = [ i.replace('\n','') for i in genes]
            dict[genes[1]] = genes[3:]

    return dict

gene_data = load_data_dict('./data/ReactomePathways.gmt')


# Raw data preprocessing and multi omics data intergration

In [14]:
# loading snv data
snv_data = pd.read_csv("./data/TCGA-LUAD.varscan2_snv.csv",index_col = 0)


#loading cnv data
cnv_data = pd.read_csv("./data/TCGA-LUAD_cnv.csv",index_col = 0)


#loading label
response  = pd.read_csv('./data/response_paper.csv',index_col=0)

#Disrupted data set
response = response.sample(frac=1)

snv_data = snv_data.sample(frac=1)

cnv_data = cnv_data.sample(frac=1)

In [15]:
#split copy number variation data
import copy
cnv_amp = copy.deepcopy(cnv_data)

#cnv_amp
cnv_amp[cnv_amp <= 0] = 0.
cnv_amp[cnv_amp > 0 ] = 1.

#cnv_del
cnv_data[cnv_data >= 0] = 0.
cnv_data[cnv_data < 0 ] = 1.
cnv_del = cnv_data


In [16]:
print(np.array(cnv_amp.values.nonzero()).shape)
print(np.array(cnv_data.values.nonzero()).shape)
print(np.array(snv_data.values.nonzero()).shape)

(2, 453943)
(2, 317421)
(2, 162637)


In [17]:
#feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
#snv
tol_snv = snv_data.join(response,how='inner')
model = SelectKBest(chi2, k=3000)
x_data1 = model.fit_transform(tol_snv.values[:,0:-1],tol_snv.values[:,-1])
fea = model.get_support()
snv_data = snv_data.loc[:,fea]

#cnv_amp
tol_amp = cnv_amp.join(response,how='inner')
model = SelectKBest(chi2, k=3000)
x_data1 = model.fit_transform(tol_amp.values[:,0:-1],tol_amp.values[:,-1])
fea = model.get_support()
cnv_amp = cnv_amp.loc[:,fea]

#cnv_del
tol_del = cnv_del.join(response,how='inner')
model = SelectKBest(chi2, k=3000)
x_data1 = model.fit_transform(tol_del.values[:,0:-1],tol_del.values[:,-1])
fea = model.get_support()
cnv_del = cnv_del.loc[:,fea]

In [18]:
print(snv_data.shape)
print(cnv_amp.shape)
print(cnv_del.shape)

(561, 3000)
(514, 3000)
(514, 3000)


In [19]:
#loading Load pre-training data
response = pd.read_csv('./data/response.csv',index_col = 0)
snv_data = snv_data.loc[response.index]
cnv_amp = cnv_amp.loc[response.index]
cnv_del = cnv_del.loc[response.index]

In [20]:
x_list = []
y_list = []
rows_list = []
cols_list = []

data_type_list =['snv_data','cnv_amp','cnv_del']

for ind in [snv_data,cnv_amp,cnv_del]: 
    get_data = ind.join(response,how='inner')
    del get_data['response']
    
    row = get_data.index      
    col = get_data.columns     
    resp = response.loc[row]   
    
    x_list.append(ind)
    y_list.append(resp)
    rows_list.append(row)
    cols_list.append(col)

In [21]:
#Merge data set
def combine(x_list, y_list, rows_list, cols_list, data_type_list, combine_type, use_coding_genes_only=True):
    
    cols_list_set = [set(list(c)) for c in cols_list]  
    
    print('cols_list_set',len(cols_list_set))

    if combine_type == 'intersection':    
        cols = set.intersection(*cols_list_set)
    else:
        cols = set.union(*cols_list_set) 
    
    print('intersection_cols',len(cols))

    if use_coding_genes_only: #true
        coding_genes_df = pd.read_csv('./data/protein-coding_gene_with_coordinate_minimal.txt', sep='\t', header=None)
        coding_genes_df.columns = ['chr', 'start', 'end', 'name']
        coding_genes = set(coding_genes_df['name'].unique())     
        cols = cols.intersection(coding_genes)  
        print('protein-coding_genes',len(coding_genes))
   
    print('finally_cols',len(cols))
   
    all_cols = list(cols)

    all_cols_df = pd.DataFrame(index=all_cols) 

    df_list = []

    for x, y, r, c in zip(x_list, y_list, rows_list, cols_list):
        df = pd.DataFrame(x, columns=c, index=r)
        df = df.T.join(all_cols_df, how='right')  
        df = df.T
        df = df.fillna(0)
        df_list.append(df)

    all_data = pd.concat(df_list, keys=data_type_list, join='inner', axis=1, )  

   
    all_data = all_data.swaplevel(i=0, j=1, axis=1)


    order = all_data.columns.levels[0] 

    all_data = all_data.reindex(columns=order, level=0)  

    x = all_data

    reordering_df = pd.DataFrame(index=all_data.index)  
    y = reordering_df.join(y, how='left')   

    y = y.values   
    cols = all_data.columns   
    rows = all_data.index      
    print(
        'After combining, loaded data %d samples, %d variables, %d responses ' % (x.shape[0], x.shape[1], y.shape[0]))

    return x, y, rows, cols

In [22]:
x, y, rows, cols = combine(x_list, y_list, rows_list, cols_list, data_type_list, combine_type = 'union')

cols_list_set 3
intersection_cols 7722
protein-coding_genes 19045
finally_cols 7034
After combining, loaded data 505 samples, 21102 variables, 505 responses 


In [23]:
rows

Index(['TCGA-55-8206', 'TCGA-05-4382', 'TCGA-69-7761', 'TCGA-50-5944',
       'TCGA-44-A479', 'TCGA-55-8620', 'TCGA-91-A4BC', 'TCGA-78-7536',
       'TCGA-55-8087', 'TCGA-55-7907',
       ...
       'TCGA-71-6725', 'TCGA-44-8117', 'TCGA-91-8499', 'TCGA-73-4659',
       'TCGA-75-6212', 'TCGA-97-8176', 'TCGA-86-8280', 'TCGA-55-6543',
       'TCGA-50-6594', 'TCGA-38-4632'],
      dtype='object', length=505)

# Annotated relationships between genes and pathways

In [24]:
pathways  = list(gene_data.keys())  
pathway_union = list(set(Get_Node_relation[3].index).intersection(set(pathways)))

print(len(pathway_union))
Get_Node_relation[3] = Get_Node_relation[3].loc[pathway_union]

union_gene = list(cols.levels[0])
len(union_gene)

550


7034

In [25]:
pathways_gene = np.zeros((len(pathway_union), len(union_gene))) 

for p  in pathway_union:
    gs = gene_data[p]      
    g_inds = [union_gene.index(g) for g in gs if g in union_gene]  
    p_ind = pathway_union.index(p)
    pathways_gene[p_ind, g_inds] = 1
gene_pathway_df = pd.DataFrame(pathways_gene, index=pathway_union, columns=union_gene)


#Drop genes that are not in the pathway
gene_pathway_df = gene_pathway_df.loc[:, (gene_pathway_df != 0).any(axis=0)]

In [26]:
print(gene_pathway_df.shape)
gene_pathway_df.head()

(550, 1542)


,ADAMTS1,YWHAQ,CTNND1,RCC2,BDKRB2,PANK3,SPP1,CRMP1,PHC3,PIP4K2A,...,GLP1R,CKAP5,PSENEN,NPFFR2,SLC22A6,OSM,LRTOMT,ACAD9,TATDN2,MAP1LC3B
R-HSA-72689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-3560783,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-1482922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-111461,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-3304356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
Omics_data = x[list(gene_pathway_df.columns)] 
Omics_data.shape

(505, 4626)

In [28]:
cols = gene_pathway_df.columns  
mapp = gene_pathway_df.values          

In [29]:
mapp

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Model construction

In [30]:
from keras.layers import Dense, Dropout, Activation, BatchNormalization, multiply
from keras.regularizers import l2
from keras import Input
from keras.engine import Model
from keras import backend as K

In [31]:
import keras
import numpy as np
from keras import regularizers
from keras.engine import Layer
# from keras import initializations
from keras.initializers import glorot_uniform, Initializer
from keras.layers import activations, initializers, constraints
# our layer will take input shape (nb_samples, 1)
from keras.regularizers import Regularizer
import tensorflow as tf

In [32]:
class M_Nets(Layer):   
    def __init__(self, units, activation=None,
                 use_bias=True,
                 kernel_initializer='lecun_uniform',
                 bias_initializer='zeros',
                 W_regularizer=None,
                 b_regularizer=None,
                 **kwargs):
        self.units = units
        self.activation = activation
        self.activation_fn = activations.get(activation)
        self.use_bias = use_bias
        self.bias_initializer = initializers.get(bias_initializer)
        self.bias_regularize = regularizers.get(b_regularizer)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.kernel_regularizer = regularizers.get(W_regularizer)
        
        super(M_Nets, self).__init__(**kwargs)


    def build(self, input_shape):  

        input_dimension = input_shape[1]
        self.kernel_shape = (input_dimension, self.units)  
        self.n_inputs_per_node = input_dimension / self.units

        rows = np.arange(input_dimension) 
        cols = np.arange(self.units)    
        cols = np.repeat(cols, self.n_inputs_per_node) 
        self.nonzero_ind = np.column_stack((rows, cols)) 

        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_dimension,),  
                                      initializer=self.kernel_initializer,
                                      regularizer=self.kernel_regularizer,
                                      trainable=True)

        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        trainable=True
                                        
                                       )
        else:
            self.bias = None

        super(M_Nets, self).build(input_shape)  

    def call(self, x, mask=None):
        
        n_features = x.shape[1]


        kernel = K.reshape(self.kernel, (1, n_features))
        mult = x * kernel
        mult = K.reshape(mult, (-1, int(self.n_inputs_per_node)))
        mult = K.sum(mult, axis=1)
        output = K.reshape(mult, (-1, self.units))

        if self.use_bias:
            output = K.bias_add(output, self.bias)
        if self.activation_fn is not None:
            output = self.activation_fn(output)
        return output

    def get_config(self):
        config = {
            'units': self.units,
            'activation': self.activation,
            'kernel_initializer' : self.kernel_initializer,
            'bias_initializer' : self.bias_initializer,
            'use_bias': self.use_bias
        }
        base_config = super(M_Nets, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

In [33]:
class Nets(Layer):
    def __init__(self, units, mapp=None, nonzero_ind=None, kernel_initializer='glorot_uniform', W_regularizer=None,
                 activation='tanh', use_bias=True,bias_initializer='zeros', bias_regularizer=None,
                 bias_constraint=None,**kwargs):
        
        self.units = units
        self.activation = activation
        self.mapp = mapp
        self.nonzero_ind = nonzero_ind
        self.use_bias = use_bias
        
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.kernel_regularizer = regularizers.get(W_regularizer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activation_fn = activations.get(activation)
        super(Nets, self).__init__(**kwargs)

        
    def build(self, input_shape):
        
        input_dim = input_shape[1]
   

        if not self.mapp is None:
            self.mapp = self.mapp.astype(np.float32)

   
        if self.nonzero_ind is None:
            nonzero_ind = np.array(np.nonzero(self.mapp)).T
            self.nonzero_ind = nonzero_ind

        self.kernel_shape = (input_dim, self.units)
        

        nonzero_count = self.nonzero_ind.shape[0]  


        self.kernel_vector = self.add_weight(name='kernel_vector',
                                             shape=(nonzero_count,),
                                             initializer=self.kernel_initializer,
                                             regularizer=self.kernel_regularizer,
                                             trainable=True)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer
                                        )
        else:
            self.bias = None

        super(Nets, self).build(input_shape)  
      

    def call(self, inputs):
        
        
        temp_t = tf.scatter_nd(tf.constant(self.nonzero_ind, tf.int32), self.kernel_vector,
                           tf.constant(list(self.kernel_shape)))
    
        output = K.dot(inputs, temp_t)
        
    
        if self.use_bias:
            output = K.bias_add(output, self.bias)
            
        if self.activation_fn is not None:
            output = self.activation_fn(output)

        return output

    def get_config(self):
        config = {
            'units': self.units,
            'activation': self.activation,
            'use_bias': self.use_bias,
            'nonzero_ind': np.array(self.nonzero_ind),
          
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),

            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'W_regularizer': regularizers.serialize(self.kernel_regularizer),

        }
        base_config = super(Nets, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))



    def compute_output_shape(self, input_shape):
      
        return (input_shape[0], self.units)


In [34]:
def create_model(Omics_data):
    
    S_inputs = Input(shape=(Omics_data.shape[1],), dtype='float32',name= 'inputs')

    h_0 = M_Nets(cols.shape[0], activation='tanh',name='h_0')(S_inputs)
    drop_layer1 =keras.layers.Dropout(0.5)(h_0)

    h0 = Nets(gene_pathway_df.shape[0],mapp =gene_pathway_df.values.T, name = 'h0')(drop_layer1)
    drop0 = keras.layers.Dropout(0.1)(h0)

    h1 = Nets(Get_Node_relation[3].shape[1],mapp =Get_Node_relation[3].values ,name = 'h1')(drop0)
    drop_h1 = keras.layers.Dropout(0.1)(h1)


    h2 = Nets(Get_Node_relation[2].shape[1],mapp =Get_Node_relation[2].values, name = 'h2')(drop_h1)
    drop2 = keras.layers.Dropout(0.1)(h2)


    h3 = Nets(Get_Node_relation[1].shape[1],mapp =Get_Node_relation[1].values, name = 'h3')(drop2)
    drop3 = keras.layers.Dropout(0.1)(h3)

    
    h4 = Nets(Get_Node_relation[0].shape[1],mapp =Get_Node_relation[0].values, name = 'h4')(drop3)
    drop4 = keras.layers.Dropout(0.1)(h4)

    Output = keras.layers.Dense(1,activation='sigmoid')(drop4)

    model = Model(inputs=S_inputs, outputs=Output)

    model.summary()

    opt = keras.optimizers.Adam(lr = 0.001) #,decay=-0.0001 ,decay=0.001
    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=['acc'])
    return model

In [44]:

import sklearn
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
   

from sklearn.metrics import precision_recall_curve
def evaluates(y_test, y_pred):
    
    auc = metrics.roc_auc_score(y_test,y_pred)
    
    aupr = average_precision_score(y_test, y_pred)
    
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred)    
    auprc  = metrics.auc(recall, precision)
    
    pp = [1 if index>=0.5  else 0 for index in  y_pred ]
    
    pre = metrics.precision_score(y_test,pp)
    
    f1 = metrics.f1_score(y_test,pp)
    
    rec = metrics.recall_score(y_test,pp)
    
    acc = metrics.accuracy_score(y_test,pp)
    
    print(confusion_matrix(y_test,pp))
    
    return pre,acc,rec,f1,auc,aupr,auprc


In [37]:
from deepexplain.model_utils import get_layers, get_coef_importance

def get_coef_importances(model, X_train, y_train, target=-1, feature_importance='deepexplain_grad*input'):

    coef_ = get_coef_importance(model, X_train, y_train, target, feature_importance, detailed=False)
    return coef_

In [38]:
from keras.callbacks import LearningRateScheduler
def myScheduler(epoch):

    if epoch % 150 == 0 and epoch != 0:

        lr = K.get_value(model.optimizer.lr)

        K.set_value(model.optimizer.lr, lr * 0.5)
    return K.get_value(model.optimizer.lr)
 
myReduce_lr = LearningRateScheduler(myScheduler)
 

In [86]:
# Omics_data.columns

In [54]:
#Five-fold cross validation

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=10590) 


kfscore = []

p = 0
x = Omics_data.values
y = y.reshape(-1)
x_0 =  0.68
x_1 =  1.48

for train_index, test_index in skf.split(x, y):

    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = create_model(x)

    history = model.fit(X_train,y_train,validation_data=[X_test,y_test],epochs=120,class_weight={0:x_0,1:x_1},batch_size = 32)  #epochs=50,class_weight={0:0.5,1:1.35},batch_size = 64


    y_pred = model.predict(X_test)
    kfscore.append(evaluates(y_test, y_pred))
    results = evaluates(y_test, y_pred)
    print("results : pre = {}, acc = {},rec = {},f1 = {},auc = {},aupr = {},auprc = {}".format(round(results[0],3),round(results[1],3),round(results[2],3),round(results[3],3),round(results[4],3),round(results[5],3),round(results[6],3)))

    # feature importance
    explain_x = X_train[np.where(y_train!=0)]
    explain_y = y_train[np.where(y_train!=0)]
    coef_ = get_coef_importance(model,explain_x, explain_y, target=-1,feature_importance='deepexplain_deeplift')
    cof_values = ['inputs','h_0','h0','h1','h2','h3','h4']
    name = [np.array(Omics_data.columns),gene_pathway_df.columns,Get_Node_relation[3].index,Get_Node_relation[2].index,Get_Node_relation[1].index,Get_Node_relation[0].index,Get_Node_relation[0].columns]
#     os.mkdir('./data/coef/h{}/'.format(p))
    for i in range(0,7):
        X = pd.DataFrame()
        X['name'] = name[i]
        X['values'] = coef_[0][cof_values[i]]
        X.to_csv('./data/coef/h{}/{}.csv'.format(p,cof_values[i]),index=False,encoding='UTF-8')
    p =p+1
#avrrage
kfscores = np.array(kfscore).sum(axis= 0)/5.0
print("average value : pre = {}, acc = {},rec = {},f1 = {},auc = {},aupr = {},auprc = {}".format(round(kfscores[0],3),round(kfscores[1],3),round(kfscores[2],3),round(kfscores[3],3),round(kfscores[4],3),round(kfscores[5],3),round(results[6],3)))



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 4626)              0         
_________________________________________________________________
h_0 (M_Nets)                 (None, 1542)              6168      
_________________________________________________________________
dropout_157 (Dropout)        (None, 1542)              0         
_________________________________________________________________
h0 (Nets)                    (None, 550)               4844      
_________________________________________________________________
dropout_158 (Dropout)        (None, 550)               0         
_________________________________________________________________
h1 (Nets)                    (None, 243)               1775      
_________________________________________________________________
dropout_159 (Dropout)        (None, 243)               0         
__________

404/404 [==============================] - 1s 3ms/step - loss: 0.3966 - acc: 0.7995 - val_loss: 0.4150 - val_acc: 0.8515
Epoch 46/120
404/404 [==============================] - 2s 4ms/step - loss: 0.3876 - acc: 0.8144 - val_loss: 0.4042 - val_acc: 0.8317
Epoch 47/120
404/404 [==============================] - 1s 3ms/step - loss: 0.3759 - acc: 0.8020 - val_loss: 0.4067 - val_acc: 0.8416
Epoch 48/120
404/404 [==============================] - 2s 4ms/step - loss: 0.3638 - acc: 0.8144 - val_loss: 0.4060 - val_acc: 0.8416
Epoch 49/120
404/404 [==============================] - 1s 3ms/step - loss: 0.3657 - acc: 0.8465 - val_loss: 0.4199 - val_acc: 0.8119
Epoch 50/120
404/404 [==============================] - 2s 4ms/step - loss: 0.4014 - acc: 0.7797 - val_loss: 0.4176 - val_acc: 0.7921
Epoch 51/120
404/404 [==============================] - 2s 5ms/step - loss: 0.3666 - acc: 0.8045 - val_loss: 0.4454 - val_acc: 0.7426
Epoch 52/120
404/404 [==============================] - 1s 3ms/step - loss:

Epoch 106/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2335 - acc: 0.8911 - val_loss: 0.3502 - val_acc: 0.8812
Epoch 107/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2445 - acc: 0.9010 - val_loss: 0.3468 - val_acc: 0.8911
Epoch 108/120
404/404 [==============================] - 1s 3ms/step - loss: 0.2259 - acc: 0.8663 - val_loss: 0.3405 - val_acc: 0.8911
Epoch 109/120
404/404 [==============================] - 1s 3ms/step - loss: 0.2176 - acc: 0.8936 - val_loss: 0.3365 - val_acc: 0.8812
Epoch 110/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2069 - acc: 0.9059 - val_loss: 0.3470 - val_acc: 0.8713
Epoch 111/120
404/404 [==============================] - 1s 3ms/step - loss: 0.2508 - acc: 0.8614 - val_loss: 0.3381 - val_acc: 0.8911
Epoch 112/120
404/404 [==============================] - 1s 3ms/step - loss: 0.2322 - acc: 0.8837 - val_loss: 0.3543 - val_acc: 0.8317
Epoch 113/120
404/404 [==============================] 

404/404 [==============================] - 1s 3ms/step - loss: 0.5775 - acc: 0.7599 - val_loss: 0.6330 - val_acc: 0.7426
Epoch 30/120
404/404 [==============================] - 1s 2ms/step - loss: 0.5624 - acc: 0.7624 - val_loss: 0.6059 - val_acc: 0.7426
Epoch 31/120
404/404 [==============================] - ETA: 0s - loss: 0.5517 - acc: 0.752 - 1s 3ms/step - loss: 0.5521 - acc: 0.7500 - val_loss: 0.6379 - val_acc: 0.7921
Epoch 32/120
404/404 [==============================] - 1s 3ms/step - loss: 0.5350 - acc: 0.7673 - val_loss: 0.6022 - val_acc: 0.7921
Epoch 33/120
404/404 [==============================] - 1s 2ms/step - loss: 0.5197 - acc: 0.7970 - val_loss: 0.6428 - val_acc: 0.6238
Epoch 34/120
404/404 [==============================] - 1s 2ms/step - loss: 0.5250 - acc: 0.6287 - val_loss: 0.5174 - val_acc: 0.7624
Epoch 35/120
404/404 [==============================] - 1s 2ms/step - loss: 0.5222 - acc: 0.7673 - val_loss: 0.7549 - val_acc: 0.4158
Epoch 36/120
404/404 [===============

Epoch 90/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2151 - acc: 0.8960 - val_loss: 0.4446 - val_acc: 0.8416
Epoch 91/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2309 - acc: 0.8713 - val_loss: 0.4464 - val_acc: 0.8416
Epoch 92/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2918 - acc: 0.8515 - val_loss: 0.4561 - val_acc: 0.8416
Epoch 93/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2719 - acc: 0.8441 - val_loss: 0.4506 - val_acc: 0.8416
Epoch 94/120
404/404 [==============================] - 1s 2ms/step - loss: 0.3015 - acc: 0.8441 - val_loss: 0.5962 - val_acc: 0.7228
Epoch 95/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2436 - acc: 0.8787 - val_loss: 0.4806 - val_acc: 0.8119
Epoch 96/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2188 - acc: 0.8861 - val_loss: 0.4751 - val_acc: 0.8119
Epoch 97/120
404/404 [==============================] - 1s 2ms

...

404/404 [==============================] - 1s 2ms/step - loss: 0.2656 - acc: 0.8837 - val_loss: 0.4509 - val_acc: 0.7822
Epoch 75/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2865 - acc: 0.8639 - val_loss: 0.4909 - val_acc: 0.7426
Epoch 76/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2459 - acc: 0.8762 - val_loss: 0.4290 - val_acc: 0.8119
Epoch 77/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2786 - acc: 0.8589 - val_loss: 0.4262 - val_acc: 0.8317
Epoch 78/120
404/404 [==============================] - 1s 2ms/step - loss: 0.3020 - acc: 0.8614 - val_loss: 0.4488 - val_acc: 0.7822
Epoch 79/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2781 - acc: 0.8713 - val_loss: 0.5802 - val_acc: 0.6634
Epoch 80/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2666 - acc: 0.8490 - val_loss: 0.4868 - val_acc: 0.7426
Epoch 81/120
404/404 [==============================] - 1s 2ms/step - loss:

Train on 404 samples, validate on 101 samples
Epoch 1/120
404/404 [==============================] - 12s 29ms/step - loss: 0.6167 - acc: 0.5743 - val_loss: 0.6754 - val_acc: 0.7327
Epoch 2/120
404/404 [==============================] - 1s 2ms/step - loss: 0.6127 - acc: 0.7351 - val_loss: 0.6520 - val_acc: 0.7327
Epoch 3/120
404/404 [==============================] - 1s 2ms/step - loss: 0.6134 - acc: 0.7351 - val_loss: 0.6371 - val_acc: 0.7327
Epoch 4/120
404/404 [==============================] - 1s 2ms/step - loss: 0.6115 - acc: 0.7351 - val_loss: 0.6441 - val_acc: 0.7327
Epoch 5/120
404/404 [==============================] - 1s 2ms/step - loss: 0.6125 - acc: 0.7351 - val_loss: 0.6432 - val_acc: 0.7327
Epoch 6/120
404/404 [==============================] - 1s 2ms/step - loss: 0.6128 - acc: 0.7351 - val_loss: 0.6456 - val_acc: 0.7327
Epoch 7/120
404/404 [==============================] - 1s 2ms/step - loss: 0.6118 - acc: 0.7351 - val_loss: 0.6530 - val_acc: 0.7327
Epoch 8/120
404/404 [

404/404 [==============================] - 1s 2ms/step - loss: 0.3264 - acc: 0.8094 - val_loss: 0.3830 - val_acc: 0.8020
Epoch 62/120
404/404 [==============================] - 1s 2ms/step - loss: 0.3372 - acc: 0.8416 - val_loss: 0.3592 - val_acc: 0.8020
Epoch 63/120
404/404 [==============================] - 1s 2ms/step - loss: 0.3277 - acc: 0.8292 - val_loss: 0.3115 - val_acc: 0.8713
Epoch 64/120
404/404 [==============================] - 1s 2ms/step - loss: 0.3502 - acc: 0.8193 - val_loss: 0.3195 - val_acc: 0.8515
Epoch 65/120
404/404 [==============================] - 1s 2ms/step - loss: 0.3313 - acc: 0.8465 - val_loss: 0.3130 - val_acc: 0.8713
Epoch 66/120
404/404 [==============================] - 1s 2ms/step - loss: 0.3402 - acc: 0.8441 - val_loss: 0.3051 - val_acc: 0.9109
Epoch 67/120
404/404 [==============================] - 1s 2ms/step - loss: 0.3014 - acc: 0.8589 - val_loss: 0.3009 - val_acc: 0.8416
Epoch 68/120
404/404 [==============================] - 1s 2ms/step - loss:

Train on 404 samples, validate on 101 samples
Epoch 1/120
404/404 [==============================] - 12s 30ms/step - loss: 0.6159 - acc: 0.7228 - val_loss: 0.6629 - val_acc: 0.7327
Epoch 2/120
404/404 [==============================] - 1s 2ms/step - loss: 0.6119 - acc: 0.7351 - val_loss: 0.6479 - val_acc: 0.7327
Epoch 3/120
404/404 [==============================] - 1s 2ms/step - loss: 0.6112 - acc: 0.7351 - val_loss: 0.6472 - val_acc: 0.7327
Epoch 4/120
404/404 [==============================] - 1s 2ms/step - loss: 0.6116 - acc: 0.7351 - val_loss: 0.6417 - val_acc: 0.7327
Epoch 5/120
404/404 [==============================] - 1s 2ms/step - loss: 0.6117 - acc: 0.7351 - val_loss: 0.6411 - val_acc: 0.7327
Epoch 6/120
404/404 [==============================] - 1s 2ms/step - loss: 0.6117 - acc: 0.7351 - val_loss: 0.6430 - val_acc: 0.7327
Epoch 7/120
404/404 [==============================] - 1s 2ms/step - loss: 0.6104 - acc: 0.7351 - val_loss: 0.6421 - val_acc: 0.7327
Epoch 8/120
404/404 [

Epoch 62/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2665 - acc: 0.8861 - val_loss: 0.4437 - val_acc: 0.8119
Epoch 63/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2818 - acc: 0.8861 - val_loss: 0.4915 - val_acc: 0.8119
Epoch 64/120
404/404 [==============================] - 1s 2ms/step - loss: 0.3165 - acc: 0.8589 - val_loss: 0.4653 - val_acc: 0.8218
Epoch 65/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2928 - acc: 0.8515 - val_loss: 0.4568 - val_acc: 0.8218
Epoch 66/120
404/404 [==============================] - 1s 2ms/step - loss: 0.3092 - acc: 0.8515 - val_loss: 0.4244 - val_acc: 0.7921
Epoch 67/120
404/404 [==============================] - 1s 2ms/step - loss: 0.2696 - acc: 0.8639 - val_loss: 0.4297 - val_acc: 0.7822
Epoch 68/120
404/404 [==============================] - 1s 2ms/step - loss: 0.3056 - acc: 0.8663 - val_loss: 0.4950 - val_acc: 0.8218
Epoch 69/120
404/404 [==============================] - 1s 2ms

In [ ]:
#  average  five result
file_name = ['inputs.csv','h_0.csv','h0.csv','h1.csv','h2.csv','h3.csv','h4.csv']
os.mkdir('./data/coef/average')
for j in file_name:
    result0 =pd.DataFrame()
    for i in range(0,5):
         result  = pd.read_csv('./data/coef/h{}/{}'.format(i,j))
         result0 = result0.append(result)
    results = pd.DataFrame(result0.groupby('name')['values'].mean()).reset_index().sort_values('values',ascending=False)
    results.to_csv('./data/coef/average/{}'.format(j),index = False)
    print(results.shape)

# single omics training

In [46]:
def create_single_model(omics_data):
    S_inputs = Input(shape=(omics_data.shape[1],), dtype='float32',name= 'inputs')
    #gene and pathway

    h0 = SparseTF(gene_pathway_df.shape[0],mapp =gene_pathway_df.values.T, name = 'h0')(S_inputs)
    drop0 = keras.layers.Dropout(0.5)(h0)

    h1 = SparseTF(Get_Node_relation[3].shape[1],mapp =Get_Node_relation[3].values ,name = 'h1')(drop0)
    drop_h1 = keras.layers.Dropout(0.1)(h1)


    h2 = SparseTF(Get_Node_relation[2].shape[1],mapp =Get_Node_relation[2].values, name = 'h2')(drop_h1)
    drop2 = keras.layers.Dropout(0.1)(h2)


    h3 = SparseTF(Get_Node_relation[1].shape[1],mapp =Get_Node_relation[1].values, name = 'h3')(drop2)
    drop3 = keras.layers.Dropout(0.1)(h3)

    
    h4 = SparseTF(Get_Node_relation[0].shape[1],mapp =Get_Node_relation[0].values, name = 'h4')(drop3)
    drop4 = keras.layers.Dropout(0.1)(h4)

    Output = keras.layers.Dense(1,activation='sigmoid')(drop4)

    model = Model(inputs=S_inputs, outputs=Output)

    model.summary()

    opt = keras.optimizers.Adam(lr = 0.001) #,decay=-0.0001
    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=['acc'])
    return model

In [47]:
#Single omics
#snv_data; cnv_amp; cnv_del
single_snv =Omics_data.swaplevel(i=0, j=1, axis=1)['snv_data']
single_snv = single_snv.join(response,how='inner')
single_snv.shape

single_x = single_snv.values[:,0:-1]
single_y =  single_snv.values[:,-1]

In [48]:
import random
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=10590)

# random.seed(5)
# total_score = []
# for i in range(0,5):
kfscore = []
for train_index, test_index in skf.split(single_x, single_y):
    X_train, X_test = single_x[train_index], single_x[test_index]
    y_train, y_test = single_y[train_index], single_y[test_index]

    single_model = create_model(single_x)

    single_model.fit(X_train,y_train,validation_data=[X_test,y_test],epochs=120,class_weight={0:0.68,1:1.48},batch_size = 32)  #epochs=50,class_weight={0:0.5,1:1.35},batch_size = 64
    y_pred = single_model.predict(X_test)
    kfscore.append(evaluates(y_test, y_pred))
    results = evaluates(y_test, y_pred)
    print("results : acc = {},rec = {},f1 = {},auc = {},aupr = {},auprc = {}".format(round(results[0],3),round(results[1],3),round(results[2],3),round(results[3],3),round(results[4],3),round(results[5],3)))


#average
kfscores = np.array(kfscore).sum(axis= 0)/5.0
print("average value : acc = {},rec = {},f1 = {},auc = {},aupr = {},auprc = {}".format(round(kfscores[0],3),round(kfscores[1],3),round(kfscores[2],3),round(kfscores[3],3),round(kfscores[4],3),round(kfscores[5],3)))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1542)              0         
_________________________________________________________________
h_0 (M_Nets)                 (None, 1542)              3084      
_________________________________________________________________
dropout_97 (Dropout)         (None, 1542)              0         
_________________________________________________________________
h0 (Nets)                    (None, 550)               4844      
_________________________________________________________________
dropout_98 (Dropout)         (None, 550)               0         
_________________________________________________________________
h1 (Nets)                    (None, 243)               1775      
_________________________________________________________________
dropout_99 (Dropout)         (None, 243)               0         
__________

404/404 [==============================] - 1s 1ms/step - loss: 0.6096 - acc: 0.7327 - val_loss: 0.6349 - val_acc: 0.7426
Epoch 46/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6040 - acc: 0.7376 - val_loss: 0.6367 - val_acc: 0.7426
Epoch 47/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6046 - acc: 0.7401 - val_loss: 0.6334 - val_acc: 0.7426
Epoch 48/120
404/404 [==============================] - 1s 1ms/step - loss: 0.5988 - acc: 0.7401 - val_loss: 0.6219 - val_acc: 0.7426
Epoch 49/120
404/404 [==============================] - 1s 1ms/step - loss: 0.5992 - acc: 0.7401 - val_loss: 0.6317 - val_acc: 0.7525
Epoch 50/120
404/404 [==============================] - 1s 1ms/step - loss: 0.5987 - acc: 0.7079 - val_loss: 0.6352 - val_acc: 0.7525
Epoch 51/120
404/404 [==============================] - 1s 1ms/step - loss: 0.5967 - acc: 0.7327 - val_loss: 0.6136 - val_acc: 0.7525
Epoch 52/120
404/404 [==============================] - 1s 1ms/step - loss:

404/404 [==============================] - 1s 1ms/step - loss: 0.3989 - acc: 0.7822 - val_loss: 0.3776 - val_acc: 0.8416
Epoch 107/120
404/404 [==============================] - 1s 1ms/step - loss: 0.3835 - acc: 0.7797 - val_loss: 0.3713 - val_acc: 0.8317
Epoch 108/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4097 - acc: 0.7847 - val_loss: 0.4364 - val_acc: 0.8614
Epoch 109/120
404/404 [==============================] - 1s 1ms/step - loss: 0.3944 - acc: 0.7797 - val_loss: 0.3919 - val_acc: 0.8812
Epoch 110/120
404/404 [==============================] - 1s 1ms/step - loss: 0.3887 - acc: 0.8069 - val_loss: 0.4155 - val_acc: 0.8713
Epoch 111/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4186 - acc: 0.7921 - val_loss: 0.5708 - val_acc: 0.6337
Epoch 112/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4323 - acc: 0.7698 - val_loss: 0.3760 - val_acc: 0.8416
Epoch 113/120
404/404 [==============================] - 1s 1ms/step 

404/404 [==============================] - 1s 1ms/step - loss: 0.6092 - acc: 0.7351 - val_loss: 0.6370 - val_acc: 0.7327
Epoch 30/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6062 - acc: 0.7351 - val_loss: 0.6408 - val_acc: 0.7327
Epoch 31/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6056 - acc: 0.7376 - val_loss: 0.6421 - val_acc: 0.7327
Epoch 32/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6028 - acc: 0.7376 - val_loss: 0.6456 - val_acc: 0.7327
Epoch 33/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6018 - acc: 0.7401 - val_loss: 0.6292 - val_acc: 0.7327
Epoch 34/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6074 - acc: 0.7376 - val_loss: 0.6186 - val_acc: 0.7327
Epoch 35/120
404/404 [==============================] - 1s 1ms/step - loss: 0.5971 - acc: 0.7550 - val_loss: 0.6369 - val_acc: 0.7327
Epoch 36/120
404/404 [==============================] - 1s 1ms/step - loss:

404/404 [==============================] - 1s 1ms/step - loss: 0.3870 - acc: 0.7896 - val_loss: 0.3644 - val_acc: 0.8416
Epoch 91/120
404/404 [==============================] - 1s 1ms/step - loss: 0.3802 - acc: 0.7921 - val_loss: 0.3872 - val_acc: 0.8020
Epoch 92/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4072 - acc: 0.8069 - val_loss: 0.3910 - val_acc: 0.8317
Epoch 93/120
404/404 [==============================] - 1s 1ms/step - loss: 0.3678 - acc: 0.8391 - val_loss: 0.3709 - val_acc: 0.8218
Epoch 94/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4195 - acc: 0.8094 - val_loss: 0.4392 - val_acc: 0.7129
Epoch 95/120
404/404 [==============================] - 1s 1ms/step - loss: 0.3857 - acc: 0.8441 - val_loss: 0.3935 - val_acc: 0.8416
Epoch 96/120
404/404 [==============================] - 1s 1ms/step - loss: 0.3898 - acc: 0.7673 - val_loss: 0.3798 - val_acc: 0.8119
Epoch 97/120
404/404 [==============================] - 1s 1ms/step - loss:

...

404/404 [==============================] - 1s 1ms/step - loss: 0.4252 - acc: 0.7797 - val_loss: 0.4485 - val_acc: 0.8020
Epoch 75/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4374 - acc: 0.7970 - val_loss: 0.4644 - val_acc: 0.8119
Epoch 76/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4273 - acc: 0.8020 - val_loss: 0.5688 - val_acc: 0.6139
Epoch 77/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4802 - acc: 0.7550 - val_loss: 0.4626 - val_acc: 0.8020
Epoch 78/120
404/404 [==============================] - 1s 2ms/step - loss: 0.4289 - acc: 0.7772 - val_loss: 0.4071 - val_acc: 0.8119
Epoch 79/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4262 - acc: 0.7376 - val_loss: 0.4036 - val_acc: 0.8119
Epoch 80/120
404/404 [==============================] - 1s 1ms/step - loss: 0.3986 - acc: 0.8193 - val_loss: 0.4262 - val_acc: 0.8020
Epoch 81/120
404/404 [==============================] - 1s 1ms/step - loss:

Train on 404 samples, validate on 101 samples
Epoch 1/120
404/404 [==============================] - 9s 23ms/step - loss: 0.6173 - acc: 0.7104 - val_loss: 0.6826 - val_acc: 0.7327
Epoch 2/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6146 - acc: 0.7351 - val_loss: 0.6658 - val_acc: 0.7327
Epoch 3/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6131 - acc: 0.7351 - val_loss: 0.6500 - val_acc: 0.7327
Epoch 4/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6138 - acc: 0.7351 - val_loss: 0.6506 - val_acc: 0.7327
Epoch 5/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6127 - acc: 0.7351 - val_loss: 0.6471 - val_acc: 0.7327
Epoch 6/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6112 - acc: 0.7351 - val_loss: 0.6456 - val_acc: 0.7327
Epoch 7/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6124 - acc: 0.7351 - val_loss: 0.6472 - val_acc: 0.7327
Epoch 8/120
404/404 [=

Epoch 62/120
404/404 [==============================] - 1s 1ms/step - loss: 0.5068 - acc: 0.7351 - val_loss: 0.5807 - val_acc: 0.6931
Epoch 63/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4888 - acc: 0.7772 - val_loss: 0.6106 - val_acc: 0.5941
Epoch 64/120
404/404 [==============================] - 1s 1ms/step - loss: 0.5011 - acc: 0.7673 - val_loss: 0.5866 - val_acc: 0.6931
Epoch 65/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4923 - acc: 0.7252 - val_loss: 0.5064 - val_acc: 0.7822
Epoch 66/120
404/404 [==============================] - ETA: 0s - loss: 0.4610 - acc: 0.826 - 1s 1ms/step - loss: 0.4662 - acc: 0.8243 - val_loss: 0.5840 - val_acc: 0.7030
Epoch 67/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4986 - acc: 0.6955 - val_loss: 0.5318 - val_acc: 0.7525
Epoch 68/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4767 - acc: 0.7624 - val_loss: 0.4999 - val_acc: 0.7822
Epoch 69/120
404/404 [==

Train on 404 samples, validate on 101 samples
Epoch 1/120
404/404 [==============================] - 9s 23ms/step - loss: 0.6164 - acc: 0.6436 - val_loss: 0.6727 - val_acc: 0.7327
Epoch 2/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6118 - acc: 0.7351 - val_loss: 0.6539 - val_acc: 0.7327
Epoch 3/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6130 - acc: 0.7351 - val_loss: 0.6440 - val_acc: 0.7327
Epoch 4/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6108 - acc: 0.7351 - val_loss: 0.6405 - val_acc: 0.7327
Epoch 5/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6117 - acc: 0.7351 - val_loss: 0.6429 - val_acc: 0.7327
Epoch 6/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6123 - acc: 0.7351 - val_loss: 0.6375 - val_acc: 0.7327
Epoch 7/120
404/404 [==============================] - 1s 1ms/step - loss: 0.6120 - acc: 0.7351 - val_loss: 0.6408 - val_acc: 0.7327
Epoch 8/120
404/404 [=

Epoch 62/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4741 - acc: 0.7772 - val_loss: 0.5413 - val_acc: 0.7624
Epoch 63/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4652 - acc: 0.7995 - val_loss: 0.5773 - val_acc: 0.7228
Epoch 64/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4979 - acc: 0.7277 - val_loss: 0.5727 - val_acc: 0.7228
Epoch 65/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4633 - acc: 0.7351 - val_loss: 0.4798 - val_acc: 0.7921
Epoch 66/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4747 - acc: 0.7574 - val_loss: 0.5519 - val_acc: 0.7426
Epoch 67/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4511 - acc: 0.8267 - val_loss: 0.5029 - val_acc: 0.7822
Epoch 68/120
404/404 [==============================] - 1s 1ms/step - loss: 0.4666 - acc: 0.7178 - val_loss: 0.4736 - val_acc: 0.7921
Epoch 69/120
404/404 [==============================] - 1s 1ms

# Other omics integration

In [49]:
snv_amp = Omics_data.swaplevel(i=0, j=1, axis=1)[['snv_data','cnv_amp']].swaplevel(i=0, j=1, axis=1)
snv_amp_order = snv_amp.columns.levels[0]
snv_amp = snv_amp.reindex(columns=snv_amp_order, level=0)
multi_x  = snv_amp.values

In [55]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=10590) 
kfscore = []
for train_index, test_index in skf.split(multi_x, y):

    X_train, X_test = multi_x[train_index], multi_x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = create_model(multi_x)

    model.fit(X_train,y_train,validation_data=[X_test,y_test],epochs=120,class_weight={0:0.68,1:1.48},batch_size = 32)  #epochs=50,class_weight={0:0.5,1:1.35},batch_size = 64
    y_pred = model.predict(X_test)
    kfscore.append(evaluates(y_test, y_pred))
    results = evaluates(y_test, y_pred)
    print("results : acc = {},rec = {},f1 = {},auc = {},aupr = {},auprc = {}".format(round(results[0],3),round(results[1],3),round(results[2],3),round(results[3],3),round(results[4],3),round(results[5],3)))

    
kfscores = np.array(kfscore).sum(axis= 0)/5.0
print("average value : acc = {},rec = {},f1 = {},auc = {},aupr = {},auprc = {}".format(round(kfscores[0],3),round(kfscores[1],3),round(kfscores[2],3),round(kfscores[3],3),round(kfscores[4],3),round(kfscores[5],3)))

In [56]:
snv_del = Omics_data.swaplevel(i=0, j=1, axis=1)[['snv_data','cnv_del']].swaplevel(i=0, j=1, axis=1)
snv_del_order = snv_del.columns.levels[0]
snv_del = snv_del.reindex(columns=snv_del_order, level=0)

In [57]:
snv_del_x  = snv_del.values

In [59]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=10590) #,shuffle=True


total_score  = []
for i in range(0,5):
    kfscore = []
    for train_index, test_index in skf.split(snv_del_x, y):

        X_train, X_test = snv_del_x[train_index], snv_del_x[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = create_model(snv_del_x)



        model.fit(X_train,y_train,validation_data=[X_test,y_test],epochs=120,class_weight={0:0.68,1:1.48},batch_size = 32)  #epochs=50,class_weight={0:0.5,1:1.35},batch_size = 64
        y_pred = model.predict(X_test)
        kfscore.append(evaluates(y_test, y_pred))
        print(evaluates(y_test, y_pred))

        temp_pd =pd.DataFrame()
        temp_pd['sample'] = sample_pro.loc[test_index]['sample'].values

        temp_pd['values'] = y_pred

        total_pd = pd.concat([total_pd,temp_pd],axis=0)

    kfscore = np.array(kfscore).sum(axis= 0)/5.0
    total_score.append(kfscore)

In [60]:
class_weight = {0:x_0,1:x_1}
compare_models = [
    {
        'type': 'sgd',
        'id': 'L2 Logistic Regression',
        'params': {'loss': 'log', 'penalty': 'l2', 'alpha': 0.01, 'class_weight': class_weight}
    },

    {
        'type': 'svc',
        'id': 'RBF Support Vector Machine ',
        'params': {'kernel': 'rbf', 'gamma': 0.001, 'probability': True, 'class_weight': class_weight}
    },

    {
        'type': 'svc', 'id':
        'Linear Support Vector Machine ',
        'params': { 'kernel': 'linear','C': 0.1, 'probability': True, 'class_weight': class_weight}  
    },

    {
        'type': 'random_forest',
        'id': 'Random Forest',
        'params': {'max_depth': None, 'n_estimators': 50, 'bootstrap': False, 'class_weight': class_weight}
    },

    {
        'type': 'adaboost',
        'id': 'Adaptive Boosting',
        'params': {'learning_rate': 0.1, 'n_estimators': 50}
    },

    {
        'type': 'decision_tree',
        'id': 'Decision Tree',
        'params': {'min_samples_split': 10, 'max_depth': 10}
    },

]

In [61]:
def get_metrics(true_score,pre_score,pre_probe):
    
    fpr, tpr, thresholds = metrics.roc_curve(true_score, pre_probe, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    
    aupr = average_precision_score(true_score, pre_probe)
    
    precision1, recall1, thresholds = precision_recall_curve(true_score, pre_probe)    
    auprc  = metrics.auc(recall1, precision1)
    
    accuracy = accuracy_score(true_score,pre_score)
    
    f1 = metrics.f1_score(true_score, pre_score)
    
    precision = metrics.precision_score(true_score,pre_score)
    
    recall = metrics.recall_score(true_score,pre_score)
    
    return accuracy,recall,f1,auc,aupr,auprc

In [62]:
from sklearn.linear_model import SGDClassifier

def Creat_SGD(whole_data_x,whole_data_y,train_index,test_index):
    
    model = SGDClassifier(**compare_models[0]['params'] )  
    
    model.fit(whole_data_x[train_index],whole_data_y[train_index] )  
    
    true_score = whole_data_y[test_index] 
    
    pre_score = model.predict(whole_data_x[test_index]) 
    
    pre_probe = model.predict_proba(whole_data_x[test_index])[:, 1] 
    

    accuracy,recall,f1,auc,aupr,auprc = get_metrics(true_score,pre_score,pre_probe) 
    
    return accuracy,recall,f1,auc,aupr,auprc

In [63]:
from sklearn.ensemble import RandomForestClassifier

def Creat_RDF(whole_data_x,whole_data_y,train_index,test_index):
     
    model = RandomForestClassifier(**compare_models[3]['params'])  

    model.fit(whole_data_x[train_index],whole_data_y[train_index] )  
    
    true_score = whole_data_y[test_index] 
    
    pre_score = model.predict(whole_data_x[test_index]) 
    
    pre_probe = model.predict_proba(whole_data_x[test_index])[:, 1] 
    

    accuracy,recall,f1,auc,aupr,auprc = get_metrics(true_score,pre_score,pre_probe) 
    
    return accuracy,recall,f1,auc,aupr,auprc


In [64]:

from sklearn.linear_model import LogisticRegression


def Creat_LR(whole_data_x,whole_data_y,train_index,test_index):
    
    model =  LogisticRegression() 

    model.fit(whole_data_x[train_index],whole_data_y[train_index] )  
    
    true_score = whole_data_y[test_index] 
    
    pre_score = model.predict(whole_data_x[test_index]) 
    
    pre_probe = model.predict_proba(whole_data_x[test_index])[:, 1] 
    

    accuracy,recall,f1,auc,aupr,auprc = get_metrics(true_score,pre_score,pre_probe) 
    
    return accuracy,recall,f1,auc,aupr,auprc

In [65]:

from sklearn import tree
 
def Creat_DTC(whole_data_x,whole_data_y,train_index,test_index):
    
    DTC_model = tree.DecisionTreeClassifier() 
    
    DTC_model.fit(whole_data_x[train_index],whole_data_y[train_index] )  
    
    true_score = whole_data_y[test_index] 
    
    pre_score = DTC_model.predict(whole_data_x[test_index]) 
    
    pre_probe = DTC_model.predict_proba(whole_data_x[test_index])[:, 1] 
    

    accuracy,recall,f1,auc,aupr,auprc = get_metrics(true_score,pre_score,pre_probe) 
    
    return accuracy,recall,f1,auc,aupr,auprc

In [66]:
from sklearn.svm import SVC, NuSVC, LinearSVC

def Creat_RBFSVM(whole_data_x,whole_data_y,train_index,test_index):

    RBFSVM = NuSVC(**compare_models[1]['params'])

    RBFSVM.fit(whole_data_x[train_index],whole_data_y[train_index] )  
    
    true_score = whole_data_y[test_index] 
    
    pre_score = RBFSVM.predict(whole_data_x[test_index]) 
    
    pre_probe = RBFSVM.predict_proba(whole_data_x[test_index])[:, 1] 
    

    accuracy,recall,f1,auc,aupr,auprc = get_metrics(true_score,pre_score,pre_probe) 
    
    return accuracy,recall,f1,auc,aupr,auprc

In [67]:
from sklearn.svm import SVC, NuSVC, LinearSVC

def Creat_LinearSVC(whole_data_x,whole_data_y,train_index,test_index):

    LSVC = SVC(**compare_models[2]['params'])

    LSVC.fit(whole_data_x[train_index],whole_data_y[train_index] )  
    
    true_score = whole_data_y[test_index] 
    
    pre_score = LSVC.predict(whole_data_x[test_index]) 
    
    pre_probe = LSVC.predict_proba(whole_data_x[test_index])[:, 1] 
    

    accuracy,recall,f1,auc,aupr,auprc = get_metrics(true_score,pre_score,pre_probe) 
    return accuracy,recall,f1,auc,aupr,auprc

In [71]:
#Comparison algorithm five-fold cross-validation
import random
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=10590) #,random_state=10590

kfscore = []

for train_index, test_index in skf.split(x,y):
         
    score = list(Creat_DTC(x,y,train_index,test_index))
    print(score)                        
    kfscore.append(score)
                                             

kfscores = np.array(kfscore).sum(axis= 0)/5.0     
print("average value : acc = {},rec = {},f1 = {},auc = {},aupr = {},auprc = {}".format(round(kfscores[0],3),round(kfscores[1],3),round(kfscores[2],3),round(kfscores[3],3),round(kfscores[4],3),round(kfscores[5],3)))

[0.6831683168316832, 0.34615384615384615, 0.35999999999999993, 0.5779487179487179, 0.30730105139969904, 0.43484189530585327]
[0.6336633663366337, 0.4074074074074074, 0.37288135593220334, 0.5395395395395396, 0.3023724579904799, 0.43969968805391185]
[0.7425742574257426, 0.37037037037037035, 0.43478260869565216, 0.6243743743743744, 0.36324860556231064, 0.5325014957636114]
[0.594059405940594, 0.3333333333333333, 0.3050847457627119, 0.5708208208208208, 0.29117892163612497, 0.42198795664590616]
[0.6633663366336634, 0.3333333333333333, 0.34615384615384615, 0.5580580580580581, 0.30446924057485114, 0.4258148672010058]
average value : acc = 0.663,rec = 0.358,f1 = 0.364,auc = 0.574,aupr = 0.314,auprc = 0.451
